In [ ]:
###This code can be used to fill in history if needed to repeat, see tradgroup_graphs_update file for just appening new data

In [1]:
import datetime
import math
import time
import itertools
from contextlib import closing
from collections import OrderedDict
import json
import numpy as np
import pandas as pd
import MySQLdb
import pymssql
import dfutils

In [2]:
DB_NAME = 'wic'
WIC_DB_HOST = 'wic-risk-database.cwi02trt7ww1.us-east-1.rds.amazonaws.com'
DB_USER = 'aduprey'
DB_PASSWORD = 'aduprey'

In [3]:
def optimized_execute(query, commit=False):
    '''
    Executes the search query in Northpoint
    '''
    
    with closing(pymssql.connect(host='10.16.1.16', user='readonly_user', password='readonly_user',
                                 database='PnLAppDb')) as np_pnl_conn:
        with closing(np_pnl_conn.cursor()) as np_pnl_cursor:
            np_pnl_cursor.execute(query)
            if commit:
                np_pnl_conn.commit()
                return

            fetched_res = np_pnl_cursor.fetchall()  # fetch (and discard) remaining rows\
            return fetched_res

In [4]:
def tradar_optimized_execute(query, commit=False):
    try:
        with closing(pymssql.connect(host='NYDC-WTRTRD01', user='paz', password='Welcome2',
                                     database='TradarBE')) as trdr_pnl_conn:
            with closing(trdr_pnl_conn.cursor()) as trdr_pnl_cursor:
                trdr_pnl_cursor.execute(query)
                if commit:
                    trdr_pnl_conn.commit()
                    return

                fetched_res = trdr_pnl_cursor.fetchall()  # fetch (and discard) remaining rows\
                return fetched_res
    except Exception as e:
        print e

In [5]:
def get_position_calculated_values_history_max_date():  # done
    query = "SELECT MAX(PCVH.TradeDate) FROM [PnLAppDb].[dbo].[PositionCalculatedValuesHistory] AS PCVH "
    results = optimized_execute(query)
    if len(results) == 0:
        return None

    return results[0][0]

In [6]:
def get_position_calculated_values_history(start_date_yyyy_mm_dd=None, end_date_yyyy_mm_dd=None,
                                           limit_to_tradegroups=[], is_tg_names_in_tradar_format=False,
                                           fund_code=None, NP_qty_rollup=True):
    if start_date_yyyy_mm_dd is None or end_date_yyyy_mm_dd is None:
        start_date_yyyy_mm_dd = '2016-01-01'
        end_date_yyyy_mm_dd = get_position_calculated_values_history_max_date()
    tg_filter_clause = ""
    if len([tg for tg in limit_to_tradegroups if tg is not None]) > 0:
        tg_filter_clause = " AND PCVH.TradeGroup IN (" + ",".join([("'" + tg + "'") 
                                                                   for tg in limit_to_tradegroups 
                                                                   if tg is not None]) + ") "
        if is_tg_names_in_tradar_format: 
            tg_filter_clause = tg_filter_clause.replace('PCVH.TradeGroup', 'SUBSTRING(PCVH.TradeGroup,0,21)')
            
    fund_code_filtuer_clause = " " if fund_code is None else " AND PCVH.FundCode = '" + fund_code + "' "
    # region query
    # New Fund by added bv Kshitij "WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL":"MACO"
    query = "SELECT " \
            "PCVH.TradeDate, " \
            "ISNULL(PCVH.FundCode, " \
            "CASE F.FundName " \
            "WHEN 'Columbia' THEN 'CAM' " \
            "WHEN 'Litman Gregory' THEN 'LG' " \
            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
            "WHEN 'TransAmerica' THEN 'TAF' " \
            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'WED' " \
            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL' THEN 'MACO' " \
            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
            "END " \
            ") AS FundCode, " \
            "SPT_TRDGRP.SecName AS TradeGroup, " \
            "SPT_TRDGRP.SecName AS TradeGroup_Tradar_Name, " \
            "PCVH.[Marketing GROUP] AS Sleeve,   " \
            "ISNULL(PCVH.[Bucket],'NA') AS Bucket, " \
            "PCVH.TradeGroupId, " \
            "SPT.Ticker, " \
            "SPT.SecType, " \
            "ISNULL(SPT.MarketCapCategory, 'N/A') AS MarketCapCategory, " \
            "PCVH.SecurityID, " \
            "PCVH.DealTermsCash, " \
            "PCVH.DealTermsStock, " \
            "PCVH.DealValue, " \
            "PCVH.DealClosingDate, " \
            "CASE " \
            "WHEN SPT.SecType = 'FxFwd' THEN 0.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Equity Special Situations'  THEN 1.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Opportunistic'  THEN 1.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Merger Arbitrage'  THEN " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') THEN " \
            "CASE PCVH.DealValue WHEN 0 THEN 0.0 ELSE PCVH.DealTermsStock/PCVH.DealValue END " \
            "ELSE 1.0 END " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Credit Opportunities'  THEN " \
            "CASE WHEN SPT.SecType IN ('EQ','ExchOpt') THEN 1.0 ELSE 0.0 END " \
            "ELSE NULL END " \
            "AS [Equity Risk Factor], " \
            "PCVH.DV_01, " \
            "PCVH.CR_01, " \
            "PCVH.Adj_CR_01, " \
            "ISNULL(SPT.UltimateCountry,'N/A') AS UltimateCountry, " \
            "ISNULL(PCVH.AlphaHedge,'NA') AS AlphaHedge, " \
            "SUM(PCVH.ExposureLong_USD) AS Exposure_Long, " \
            "SUM(PCVH.ExposureShort_USD) AS Exposure_Short, " \
            "SUM(PCVH.ExposureLong_USD+PCVH.ExposureShort_USD) AS Exposure_Net, " \
            "CASE WHEN NAV.NAV = 0 THEN NULL " \
            "ELSE 100.0*(SUM(PCVH.ExposureLong_USD+PCVH.ExposureShort_USD)/NAV.NAV) END " \
            "AS [Exposure_Net(%)], " \
            "SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD) AS NetMktVal, " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') " \
            "THEN ABS(SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD)) ELSE NULL END AS Capital, " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') " \
            "THEN 100.0*(ABS(SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD))/NAV.NAV) ELSE NULL " \
            "END AS [Capital % of NAV], " \
            "100.0*SUM(PCVH.Base_Case_NAV_Impact) AS BaseCaseNavImpact, " \
            "100.0*SUM(PCVH.Outlier_NAV_Impact) AS OutlierNavImpact, " \
            "SUM(PCVH.Qty) AS QTY, " \
            "ISNULL(PCVH.TradeGroupLongShortFlag,'NA') AS LongShort, " \
            "ISNULL(SPT.GICSSectorName,'NA') AS Sector, " \
            "ISNULL(SPT.GICSIndustryName,'NA') AS Industry, " \
            "NAV.NAV " \
            "FROM [PnLAppDb].[dbo].[PositionCalculatedValuesHistory] AS PCVH " \
            "INNER JOIN PnLAppDb.dbo.Funds AS F ON PCVH.Portfolio = F.FundID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT ON PCVH.SecurityID = SPT.ID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TRDGRP ON PCVH.TradeGroupId = SPT_TRDGRP.ID " \
            "LEFT OUTER JOIN PnLAppDb.pnl.DailyNAV AS NAV ON PCVH.Portfolio = NAV.FundId " \
            "AND PCVH.TradeDate = NAV.[DATE] " \
            "WHERE  TradeDate >= '" + start_date_yyyy_mm_dd + "' AND TradeDate <= '" + end_date_yyyy_mm_dd + "' " \
            + tg_filter_clause + \
            fund_code_filtuer_clause + \
            "GROUP BY " \
            "F.FundName, PCVH.FundCode,PCVH.TradeDate, PCVH.[Marketing GROUP], SPT_TRDGRP.SecName, " \
            "PCVH.TradeGroupId, SPT.Ticker, SPT.BloombergGlobalId, SPT.BloombergGID, " \
            "SPT.SecType, ISNULL(SPT.MarketCapCategory, 'N/A'), " \
            "PCVH.SecurityId, SPT.UltimateCountry, " \
            "ISNULL(PCVH.AlphaHedge,'NA'),PCVH.TradeGroupLongShortFlag, ISNULL(SPT.GICSSectorName,'NA'), " \
            "ISNULL(SPT.GICSIndustryName,'NA'), ISNULL(PCVH.TradeGroupCatalystRating, 'NA'), " \
            "ISNULL(PCVH.[Bucket],'NA'), PCVH.DealTermsCash, PCVH.DealTermsStock, " \
            "PCVH.DealValue, PCVH.DealClosingDate, " \
            "PCVH.DV_01, PCVH.CR_01, PCVH.Adj_CR_01, NAV.NAV, PCVH.Analyst " \
            "ORDER BY PCVH.TradeDate "
    # endregion
    results = optimized_execute(query)
    df = pd.DataFrame(results, columns=["Date", "FundCode", "TradeGroup", "TradeGroup_Tradar_Name", "Sleeve",
                                        "Bucket", "TradeGroupId", "Ticker", "SecType", "MarketCapCategory",
                                        "SecurityId", "DealTermsCash", "DealTermsStock", "DealValue", 
                                        "DealClosingDate", "Equity_Risk_Factor", "DV01", "CR01", "Adj_CR01",
                                        "UltimateCountry", "AlphaHedge", "Exposure_Long", "Exposure_Short",
                                        "Exposure_Net", "Exposure_Net(%)", "NetMktVal", "Capital($)", 
                                        "Capital(%)", "BaseCaseNavImpact", "OutlierNavImpact", "Qty", 
                                        "LongShort", "Sector", "Industry", "Fund_NAV"])

    float_cols = ['NetMktVal', 'Capital($)', 'Capital(%)', 'Exposure_Net', 'Exposure_Net(%)', 'BaseCaseNavImpact']
    df[float_cols] = df[float_cols].astype(float)
    df['Equity_Risk_Exp'] = df['Equity_Risk_Factor'].astype(float)*df['Exposure_Net'].astype(float)
    df[['DV01', 'CR01', 'Adj_CR01']] = df[['DV01', 'CR01', 'Adj_CR01']].fillna(0).astype(float)
    df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x))
    df["DealClosingDate"] = df["DealClosingDate"].apply(lambda x: pd.to_datetime(x))

    as_of_dt = datetime.datetime.strptime(end_date_yyyy_mm_dd, '%Y-%m-%d')

    def duration_calc(days):
        if days <= 90:
            return '0M-3M'
        if days <= 180:
            return '3M-6M'
        if days <= 365:
            return '6M-12M'
        return 'Yr+'

    df["DealDuration"] = df["DealClosingDate"].apply(lambda x: None if pd.isnull(x) else
                                                     duration_calc((as_of_dt - pd.to_datetime(x)).days))

    # region ADJUSTING FOR POSTED TRADES AFTER TRADE DATE - AFFECTING QTY ONLY
    # adjust for end_date_yyyy_mm_dd - take trades with tradedate = end_date_yyyy_mm_dd and posted_date
    #AFTER end_date_yyyy_mm_dd
    # adjust the Qty into the pcvh_df
    if NP_qty_rollup:
        qty_adjust_query = "SELECT " \
                            "A.TradeDate AS [DATE], " \
                            "CASE A.Fund " \
                            "WHEN 'Columbia' THEN 'CAM' " \
                            "WHEN 'Litman Gregory' THEN 'LG' " \
                            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
                            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
                            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
                            "WHEN 'TransAmerica' THEN 'TAF' " \
                            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
                            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
                            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
                            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'LEV' " \
                            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL COMMINGLED MASTER FUND LP' THEN 'MACO' " \
                            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
                            "END AS FundCode, " \
                            "A.TradeGroupId, " \
                            "A.SecurityId, " \
                            "SUM((CASE SPT.SecType WHEN 'ExchOpt' THEN 100.0 ELSE 1.0 END)*A.Shares) AS Qty " \
                            "FROM " \
                            "[PnLAppDb].[dbo].[vTradesFlatView] AS A " \
                            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT ON A.SecurityId = SPT.ID " \
                            "WHERE PostedDate >= DATEADD(DAY,1,TradeDate) AND TradeDate >= '" \
                            + start_date_yyyy_mm_dd + "' " \
                            "GROUP BY A.TradeDate, A.SecurityId, A.Ticker, A.Fund, A.TradeGroupId, SPT.SecType "
        results = optimized_execute(qty_adjust_query)
        qty_adjust_df = pd.DataFrame(results, columns=['Date', 'FundCode', 'TradeGroupId', 'SecurityId', 'Qty_adj'])
        qty_adjust_df[['TradeGroupId', 'SecurityId', 'Qty_adj']] = qty_adjust_df[['TradeGroupId',
                                                                                  'SecurityId', 'Qty_adj']].astype(float)
        qty_adjust_df['Date'] = qty_adjust_df['Date'].apply(lambda x: pd.to_datetime(x))

        df[['TradeGroupId', 'SecurityId', 'Qty']] = df[['TradeGroupId', 'SecurityId', 'Qty']].astype(float)
        df = pd.merge(df, qty_adjust_df, how='left', on=['Date', 'FundCode', 'TradeGroupId', 'SecurityId'])
        df['Qty'] = df['Qty'] + df['Qty_adj'].fillna(0)
        del df['Qty_adj']
    # endregion
    print('PCVH retrieval completed....')
    return df

In [7]:
def get_NAV_df():
    query = "SELECT A.[DATE],A.NAV, " \
            "CASE B.FundName " \
            "WHEN 'Columbia' THEN 'CAM' " \
            "WHEN 'Litman Gregory' THEN 'LG' " \
            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
            "WHEN 'TransAmerica' THEN 'TAF' " \
            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'LEV' " \
            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL' THEN 'MACO' " \
            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
            "END AS FundCode " \
            "FROM PnLAppDb.pnl.DailyNAV AS A " \
            "INNER JOIN PnLAppDb.dbo.Funds AS B ON A.FundId = B.FundID " \
            "ORDER BY A.[DATE]"

    results = optimized_execute(query)
    df = pd.DataFrame(results, columns=["Date", "NAV", "FundCode"])
    df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
    df.index.name = 'Date'
    df = df.sort_index()
    df['NAV'] = df['NAV'].astype(float)
    #idx = pd.date_range(df['Date'].min(), df['Date'].max())
    #df = df.reindex(idx, fill_value=np.nan)
    del df['Date']
    df.reset_index(level=0, inplace=True)
    df.rename(columns={"index": "Date"}, inplace=True)
    df.index.name = 'Date'
    df.ffill(inplace=True)
    df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    # df.reset_index(inplace=True)
    df = df.reset_index(level=0, drop=True).reset_index()  # Added by Kshitij
    del df['index']
    return df[df['Date'] <= (datetime.datetime.today() - datetime.timedelta(days=1))]

In [8]:
def get_spread_history():
    query = "SELECT DISTINCT " \
            "PCVH.TradeDate, " \
            "SPT_TRDGRP.SecName AS TradeGroup, " \
            "SPT_TRDGRP.SecName AS TradeGroup_TradarName, " \
            "PCVH.TradeGroupId, " \
            "PCVH.AllInSpread, " \
            "PCVH.DealValue, " \
            "100.0*(PCVH.AllInSpread/PCVH.DealValue) AS spread " \
            "FROM PnLAppDb.dbo.PositionCalculatedValuesHistory AS PCVH " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TRDGRP ON PCVH.TradeGroupId = SPT_TRDGRP.ID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TARGET ON PCVH.SecurityID= SPT_TARGET.ID " \
            "WHERE  SPT_TRDGRP.TargetTickers LIKE ('%'+SPT_TARGET.Ticker+'%') AND PCVH.TradeDate >= '2015-11-09' " \
            "AND PCVH.ConsolidatedMarketingGroup = 'Merger Arbitrage' AND PCVH.DealValue <> 0 " \
            "AND PCVH.AllInSpread IS NOT NULL " \
            "ORDER BY TradeDate "
    
    results = optimized_execute(query)
    columns = ["Date", "TradeGroup", "TradeGroup_TradarName", "TradeGroupId", "AllInSpread", "DealValue", "Spread(%)"]
    df = pd.DataFrame(results, columns=columns)
    df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x))
    return df

In [9]:
def wic_optimized_execute(query, commit=False, retrieve_column_names=False, connection_timeout=250, extra_values=None):
    with closing(MySQLdb.connect(host=WIC_DB_HOST, user=DB_USER, passwd=DB_PASSWORD, db=DB_NAME)) as wic_cnx:
        with closing(wic_cnx.cursor()) as wic_cursor:

            if extra_values is not None:
                wic_cursor.execute(query, (extra_values,))
            else:
                wic_cursor.execute(query)
            if commit:
                wic_cnx.commit()
                return
            fetched_res = wic_cursor.fetchall()  # fetch (and discard) remaining rows

            if retrieve_column_names:
                return fetched_res, [i[0] for i in wic_cursor.description]

            return fetched_res

def get_tradegroups_pnl_cache(fundcode=None, tg=None):
    query = "SELECT `DATE`, Fund, TradeGroup, pnl, cumpnl FROM " + DB_NAME + ".`tradegroups_pnl_cache`"
    if fundcode is not None and tg is not None:
        query += " WHERE Fund = '" + fundcode + "' AND TradeGroup ='" + tg + "' "
    elif fundcode is not None:
        query += " WHERE Fund = '" + fundcode + "' "
    elif tg is not None:
        query += " WHERE TradeGroup = '" + tg + "' "
    else:
        query += ""
        
    res = wic_optimized_execute(query)
    cols = ['Date', 'Fund', 'TradeGroup', 'Total P&L', 'Cumulative Total P&L']
    df = pd.DataFrame(list(res), columns=cols)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    df['Total P&L'] = df['Total P&L'].astype(float)
    df['Cumulative Total P&L'] = df['Cumulative Total P&L'].astype(float)
    return df

In [10]:
def get_securities_pnl_by_tradegroup_and_fund(limit_to_tradegroups=[], limit_to_funds=[],
                                              is_tg_names_in_tradar_format=False,
                                              start_date_yyyymmdd=None, end_date_yyyymmdd=None, rollup_pnl=False):
    if start_date_yyyymmdd is None: start_date_yyyymmdd = '20160101'
    if end_date_yyyymmdd is None:
        now = datetime.datetime.now()
        slicer = dfutils.df_slicer()
        end_date = slicer.prev_n_business_days(1, now)
        end_date_yyyymmdd = end_date.strftime('%Y%m%d')

    tg_filter_clause = ""
    if len([tg for tg in limit_to_tradegroups if tg is not None]) > 0:
        tg_filter_clause = " AND B.strat IN (" + ",".join([("'" + tg + "'")
                                                           for tg in limit_to_tradegroups 
                                                           if tg is not None]) + ") "
    if is_tg_names_in_tradar_format is True: 
        tg_filter_clause = tg_filter_clause.replace('B.strat', 'SUBSTRING(B.strat,0,21)')
    
    fund_code_filter_clause = ""
    if len([fund for fund in limit_to_funds if fund is not None]) > 0:
        fund_code_filter_clause = " AND F.fund IN (" + ",".join([("'" + fund + "'")
                                                                 for fund in limit_to_funds
                                                                 if fund is not None]) + ") "
        
    # region rollup query
    rollup_sql_query = "DECLARE @Holidays TABLE ([Date] DATE); " \
                        "DECLARE @StartDt INT; " \
                        "DECLARE @EndDt INT; " \
                        "INSERT INTO @Holidays VALUES " \
                        "('2015-01-01'), " \
                        "('2015-01-19'), " \
                        "('2015-02-16'), " \
                        "('2015-04-03'), " \
                        "('2015-05-25'), " \
                        "('2015-07-03'), " \
                        "('2015-09-07'), " \
                        "('2015-10-12'), " \
                        "('2015-11-11'), " \
                        "('2015-11-26'), " \
                        "('2015-12-25'), " \
                        "('2016-01-01'), " \
                        "('2016-01-18'), " \
                        "('2016-02-15'), " \
                        "('2016-03-25'), " \
                        "('2016-05-30'), " \
                        "('2016-07-04'), " \
                        "('2016-09-05'), " \
                        "('2016-11-24'), " \
                        "('2016-12-25'), " \
                        "('2017-01-02'), " \
                        "('2017-01-16'), " \
                        "('2017-02-20'), " \
                        "('2017-04-14'), " \
                        "('2017-05-29'), " \
                        "('2017-07-04'), " \
                        "('2017-09-04'), " \
                        "('2017-10-09'), " \
                        "('2017-11-23'), " \
                        "('2017-12-25'); " \
                        "SET @StartDt = " + start_date_yyyymmdd + "; " \
                        "SET @EndDt = " + end_date_yyyymmdd + "; " \
                        "WITH X AS( " \
                        "SELECT " \
                        "CAST(CONVERT(varchar(8), A.timeKey) AS DATE) as [Date], " \
                        "F.fund, " \
                        "B.strat AS TradeGroup, " \
                        "S.ticker, " \
                        "ST.groupingName as sectype, " \
                        "SUM(pnlFC) as PNL " \
                        "FROM performanceAttributionFact AS A " \
                        "INNER JOIN TradeStrat AS B ON A.stratKey = B.stratId " \
                        "INNER JOIN TradeFund AS F ON A.fundKey = F.fundId " \
                        "INNER JOIN Sec AS S ON S.secId = A.secIdKey " \
                        "INNER JOIN SecType AS ST ON ST.sectype = S.sectype " \
                        "WHERE A.timeKey >= @StartDt and A.timeKey <= @EndDt " \
                        + tg_filter_clause + \
                        fund_code_filter_clause + \
                        "GROUP BY A.timeKey, B.strat, F.fund, S.ticker, ST.groupingName " \
                        ") " \
                        ", PNL AS ( " \
                        "SELECT " \
                        "X.[Date],  " \
                        "CASE " \
                        "WHEN DATEPART(dw,X.[Date]) IN (7,1) THEN 'WEEKEND' " \
                        "WHEN X.[Date] IN (SELECT * FROM @Holidays) THEN 'HOLIDAY' " \
                        "ELSE 'BUSINESS DAY' END AS [WeekDay], " \
                        "X.Fund, " \
                        "X.TradeGroup, " \
                        "X.Ticker, " \
                        "X.sectype, " \
                        "X.PNL " \
                        "FROM X " \
                        "), " \
                        "Position2EndDate AS ( " \
                        "SELECT " \
                        "X.fund, " \
                        "X.TradeGroup, " \
                        "X.ticker, " \
                        "X.sectype, " \
                        "MAX(X.[Date]) AS MaxDate, " \
                        "CASE DATEPART(dw,MAX(X.[Date])) " \
                        "WHEN 6 THEN DATEADD(DAY,3,MAX(X.[Date])) " \
                        "WHEN 7 THEN DATEADD(DAY,2,MAX(X.[Date])) " \
                        "ELSE DATEADD(DAY,1,MAX(X.[Date])) END AS NextBusinessDay " \
                        "FROM PNL AS X " \
                        "GROUP BY X.fund, X.TradeGroup, X.ticker, X.sectype " \
                        "), " \
                        "Position2EndDate_BD AS ( " \
                        "SELECT " \
                        "X.fund, " \
                        "X.TradeGroup, " \
                        "X.ticker, " \
                        "X.sectype, " \
                        "X.MaxDate, " \
                        "CASE " \
                        "WHEN NextBusinessDay IN (SELECT * FROM @Holidays) THEN " \
                        "CASE DATEPART(dw,NextBusinessDay) " \
                        "WHEN 6 THEN DATEADD(DAY,3,NextBusinessDay) " \
                        "ELSE DATEADD(DAY,1,NextBusinessDay) END " \
                        "ELSE NextBusinessDay " \
                        "END AS NextBusinessDay_Holiday_Proof " \
                        "FROM Position2EndDate AS X " \
                        "), " \
                        "PNL_DT AS ( " \
                        "SELECT " \
                        "A1.[Date], " \
                        "A1.[WeekDay],  " \
                        "A1.fund, " \
                        "A1.TradeGroup, " \
                        "A1.ticker, " \
                        "A1.sectype, " \
                        "A1.PNL, " \
                        "MAX(A2.[Date]) AS [Last Business Day], " \
                        "DATEDIFF(DAY,MAX(A2.[Date]),A1.[Date]) AS [Last BD days diff] " \
                        "FROM PNL AS A1 " \
                        "LEFT OUTER JOIN PNL AS A2 ON A1.fund = A2.fund and A1.TradeGroup = A2.TradeGroup " \
                        "AND A1.ticker = A2.ticker AND A1.sectype = A2.sectype AND A2.[Date] < A1.[Date] " \
                        "AND A2.[WeekDay] = 'BUSINESS DAY' " \
                        "GROUP BY A1.[Date],A1.[WeekDay],A1.fund, A1.TradeGroup, A1.ticker, A1.sectype, A1.PNL " \
                        "), " \
                        "T AS ( " \
                        "SELECT " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN B.NextBusinessDay_Holiday_Proof ELSE A1.[Date] END AS [Date], " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN 'BUSINESS DAY' ELSE A1.[WeekDay] END AS [WeekDay], " \
                        "A1.fund, " \
                        "A1.TradeGroup, " \
                        "A1.ticker, " \
                        "A1.sectype, " \
                        "A1.[Last Business Day], " \
                        "A1.PNL, " \
                        "ISNULL(SUM(A2.PNL),0) AS ROLLED_UP_PNL, " \
                        "A1.PNL + ISNULL(SUM(A2.PNL),0) AS TOT_PNL,  " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN 'Closed on non-BD. Shifted to next BD' ELSE NULL END AS 'Notes' " \
                        "FROM PNL_DT AS A1 " \
                        "LEFT OUTER JOIN PNL_DT AS A2 " \
                        "ON A1.fund = A2.fund AND A1.TradeGroup = A2.TradeGroup AND A1.ticker = A2.ticker " \
                        "AND A1.sectype = A2.sectype AND " \
                        "(A2.[Date] < A1.[Date] AND (A2.[Date] > A1.[Last Business Day] " \
                        "OR A1.[Last Business Day] IS NULL)) " \
                        "AND (A1.[Last BD days diff] <= 4 OR A1.[Last BD days diff] IS NULL) " \
                        "INNER JOIN Position2EndDate_BD AS B ON A1.fund = B.fund AND A1.TradeGroup = B.TradeGroup " \
                        "AND A1.ticker = B.ticker AND A1.sectype = B.sectype " \
                        "WHERE A1.[WeekDay] = 'BUSINESS DAY' OR A1.[Date] = B.MaxDate " \
                        "GROUP BY A1.[Date],A1.[WeekDay], A1.fund,A1.TradeGroup, A1.ticker, " \
                        "A1.sectype, A1.[Last Business Day],A1.[Last BD days diff],A1.PNL, B.MaxDate, " \
                        "B.NextBusinessDay_Holiday_Proof " \
                        ") " \
                        "SELECT " \
                        "T.[Date],  " \
                        "T.fund, " \
                        "T.TradeGroup, " \
                        "T.ticker, " \
                        "T.sectype, " \
                        "T.TOT_PNL " \
                        "FROM T " \
                        "ORDER BY T.fund, T.TradeGroup, T.ticker,T.sectype, T.[Date] "
        
    # endregion
    # region non-rollup query
    # don't take today's pnl - tradar's garbage data
    non_rollup_sql_query = \
        "SELECT " \
        "CAST(CONVERT(VARCHAR(8), A.timeKey) AS DATE) AS [DATE], " \
        "F.fund, " \
        "B.strat AS TradeGroup, " \
        "S.ticker, " \
        "ST.groupingName AS sectype, " \
        "SUM(pnlFC) AS PNL " \
        "FROM performanceAttributionFact AS A " \
        "INNER JOIN TradeStrat AS B ON A.stratKey = B.stratId " \
        "INNER JOIN TradeFund AS F ON A.fundKey = F.fundId " \
        "INNER JOIN Sec AS S ON S.secId = A.secIdKey " \
        "INNER JOIN SecType AS ST ON ST.sectype = S.sectype " \
        "WHERE A.timeKey >= " + start_date_yyyymmdd + " AND A.timeKey <= " \
        + end_date_yyyymmdd + \
        tg_filter_clause + \
        fund_code_filter_clause + \
        "GROUP BY A.timeKey, B.strat, F.fund,S.ticker,ST.groupingName " \
        "ORDER BY F.fund, B.strat,S.ticker,A.timeKey "
    # endregion
    query = rollup_sql_query if rollup_pnl else non_rollup_sql_query
    cols = ['Date', 'Fund', 'TradeGroup', 'Ticker', 'SecType', 'Total P&L']
    try:
        query_result = tradar_optimized_execute(query)
    except:
        return pd.DataFrame(columns=cols)

    df = pd.DataFrame(query_result, columns=cols)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    df['Total P&L'] = df['Total P&L'].astype(float)
    df['Cumulative Total P&L'] = None
    
    return df

In [12]:
def perf_timeseries_df(pnl_df, capital_df, as_of_date=None, calc_stats=True, periods=('MTD', 'QTD', 'YTD',
                                                                                      'ITD', '1D', '3D',
                                                                                      '5D', '30D')):
    empty_stats_df = pd.DataFrame(columns=['Period', 'P&L($)', 'P&L(bps)', 'ROMC(bps)',
                                           'ANN. VOL', 'CAPITAL($)', 'CAPITAL CHG(%)'])
    empty_stats_df["Period"] = periods
    empty_perf_df = pd.DataFrame(columns=pnl_df.columns)
    empty_res = (empty_perf_df, empty_stats_df) if calc_stats else empty_perf_df
    if len(pnl_df) == 0 or len(pnl_df[pd.isnull(pnl_df['Total P&L'])]) > 0:
        return empty_res

    # pnl data present. at least calculate p&l cumsum
    slicer = dfutils.df_slicer(as_of_date)
    empty_stats_df['P&L($)'] = [None if len(sliced_df) == 0 else sliced_df['Total P&L'].sum()
                                for (period, sliced_df) in zip(periods, [slicer.slice(pnl_df, p) for p in periods])]

    if capital_df is None: 
        return empty_res
    if len(capital_df) == 0 or len(capital_df.columns) <= 1:
        return empty_res
    if len(capital_df[pd.isnull(capital_df[capital_df.columns[1]])]) > 0:
        return empty_res

    capital_cln = capital_df.columns[1]
    capital_df_cpy = capital_df.copy() # don't change capital_df. work on its copy
    capital_df_cpy['Capital'] = capital_df_cpy[capital_cln]
    capital_df_cpy['Shifted Capital'] = capital_df_cpy[capital_cln].shift(1)
    df = pd.merge(pnl_df, capital_df_cpy, how='left', on=['Date']).sort_values(by='Date') 
    if pd.isnull(df['Shifted Capital'].iloc[0]): df.loc[0, 'Shifted Capital'] = df['Capital'].iloc[0]
    df['Shifted Capital'] = df['Shifted Capital'].apply(lambda x: np.nan if x == 0 else x)
    # if pd.isna(df['Shifted Capital'].iloc[1]):
    #     print('Updating Shifted capital')
    #     df['Shifted Capital'].loc[1] = df['Capital'].loc[1]
    #
    df['Shifted Forward-Filled Capital'] = df['Shifted Capital'].ffill()
    # if pd.isna(df['Shifted Forward-Filled Capital'].iloc[1]):
    #     print('Updating Shifted Forward-filled Capital')
    #     df['Shifted Forward-Filled Capital'].loc[1] = df['Capital'].loc[1]
    try:
        if df['Fund'].iloc[0] == 'MACO':   # Temp Fix for MACO
            df['Shifted Capital'].iloc[1] = df['Capital'].iloc[1]
            df['Shifted Forward-Filled Capital'].iloc[1] = df['Capital'].iloc[1]
        # df.drop(df.index[0], inplace=True)
        #df.to_csv('TEMPDF-aprl3.csv')

    except Exception as err:
        print(err)
        print('Fund not found in Dataframe')

    df["P&L bps (ffiled)"] = 1e4*(df["Total P&L"]/df["Shifted Forward-Filled Capital"]).replace([np.inf,
                                                                                                 -np.inf], np.nan)
    df["Cumulative P&L ($)"] = df["Total P&L"].cumsum()
    df["Cumulative P&L bps (ffiled)"] = 1e4*((1.0+(df["P&L bps (ffiled)"].astype(float)/1e4)).cumprod() -1)
    df['Rolling 30D Vol(%)'] = math.sqrt(252)*df['P&L bps (ffiled)'].rolling(window=30).std()/100.0
    df['Rolling 60D Vol(%)'] = math.sqrt(252)*df['P&L bps (ffiled)'].rolling(window=60).std()/100.0
    df['Rolling 90D Vol(%)'] = math.sqrt(252)*df['P&L bps (ffiled)'].rolling(window=90).std()/100.0
    print "yes"
    if not calc_stats:
        return df

    # calc stats
    stats_df = pd.DataFrame(columns=["Period", 'P&L($)', 'P&L(bps)', 'ROMC(bps)',
                                     'ANN. VOL', 'CAPITAL($)', 'CAPITAL CHG(%)'])

    for period in periods:
        p_df = slicer.slice(df, period)

        next_idx = 0 if pd.isnull(stats_df.index.max()) else stats_df.index.max()+1
        if len(p_df) == 0:
            stats_df.loc[next_idx] = [period, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
            continue

        rets = p_df["P&L bps (ffiled)"].astype(float)
        vol_pct = math.sqrt(252)*np.std(rets/1e4, ddof=1)*1e2
        cumpnl_usd = p_df['Total P&L'].sum()
        cumret_bps = ((1+rets/1e4).prod() -1)*1e4
        capitals = p_df['Shifted Forward-Filled Capital'].fillna(0).astype(float)
        mean_capital = capitals.mean()
        if mean_capital == 0:
            stats_df.loc[next_idx] = [period, cumpnl_usd, cumret_bps, np.nan, vol_pct, np.nan, np.nan]
            continue
        #mw_rets = (rets/1e4)*(capitals/mean_capital)
        #romwc_bps = ((1+mw_rets).prod() -1)*1e4
        romc_bps = 1e4*(cumpnl_usd/mean_capital)
        capital_usd = capitals.iloc[-1]
        capital_chg_pct = 1e2*((capitals.iloc[-1]/capitals.iloc[0])-1.0) if capitals.iloc[0] != 0 else np.nan
        stats_df.loc[next_idx] = [period, cumpnl_usd, cumret_bps, romc_bps, vol_pct,
                                  capital_usd, capital_chg_pct]
    return df, stats_df

In [16]:
def mktval_df(pcvh_df):
    float_cols = ['Alpha Exposure', 'AlphaHedge Exposure', 'Hedge Exposure',
                  'Alpha NetMktVal', 'AlphaHedge NetMktVal', 'Hedge NetMktVal',
                  'Alpha GrossMktVal', 'AlphaHedge GrossMktVal', 'Hedge GrossMktVal',
                  'Alpha GrossExp', 'AlphaHedge GrossExp', 'Hedge GrossExp']
    calc_cols = ['Bet Exposure', 'Bet NetMktVal','Bet GrossMktVal', 'Total NetExposure',
                 'Total NetMktVal', 'Total GrossMktVal']

    if len(pcvh_df) == 0: return pd.DataFrame(columns=["Date"]+float_cols+calc_cols)

    alpha_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Alpha']
    alphahedge_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Alpha Hedge']
    hedge_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Hedge']

    alpha_net_exp = alpha_pcvh[['Date', 'Exposure_Net']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'Exposure_Net':'Alpha Exposure'})
    alpha_net_mv = alpha_pcvh[['Date', 'NetMktVal']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'Alpha NetMktVal'})
    alpha_gross_mv = alpha_pcvh[['Date', 'NetMktVal']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'NetMktVal':'Alpha GrossMktVal'})
    alpha_gross_exp = alpha_pcvh[['Date', 'Exposure_Net']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'Exposure_Net':'Alpha GrossExp'})

    alphahedge_net_exp = alphahedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'Exposure_Net':'AlphaHedge Exposure'})
    alphahedge_net_mv = alphahedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'AlphaHedge NetMktVal'})
    alphahedge_gross_mv = alphahedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'NetMktVal':'AlphaHedge GrossMktVal'})
    alphahedge_gross_exp = alphahedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'Exposure_Net':'AlphaHedge GrossExp'})

    hedge_net_exp = hedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date','Exposure_Net':'Hedge Exposure'})
    hedge_net_mv = hedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'Hedge NetMktVal'})
    hedge_gross_mv = hedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'NetMktVal':'Hedge GrossMktVal'})
    hedge_gross_exp = hedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'Exposure_Net':'Hedge GrossExp'})

    mktval_df = pd.merge(alpha_net_exp, alphahedge_net_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_net_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alpha_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alphahedge_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alpha_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alphahedge_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alpha_gross_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alphahedge_gross_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_gross_exp, how='outer', on=['Date']).fillna(0)

    mktval_df[float_cols] = mktval_df[float_cols].astype(float)
    mktval_df['Bet Exposure'] = mktval_df['Alpha Exposure']+mktval_df['AlphaHedge Exposure']
    mktval_df['Bet NetMktVal'] = mktval_df['Alpha NetMktVal']+mktval_df['AlphaHedge NetMktVal']
    mktval_df['Bet GrossMktVal'] = mktval_df['Alpha GrossMktVal']+mktval_df['AlphaHedge GrossMktVal']
    mktval_df['Total NetExposure'] = mktval_df['Alpha Exposure']+mktval_df['AlphaHedge Exposure']+mktval_df['Hedge Exposure']
    mktval_df['Total GrossExposure'] = mktval_df['Alpha GrossExp']+mktval_df['AlphaHedge GrossExp']+mktval_df['Hedge GrossExp']
    mktval_df['Total NetMktVal'] = mktval_df['Alpha NetMktVal']+mktval_df['AlphaHedge NetMktVal']+mktval_df['Hedge NetMktVal']
    mktval_df['Total GrossMktVal'] = mktval_df['Alpha GrossMktVal']+mktval_df['AlphaHedge GrossMktVal']+mktval_df['Hedge GrossMktVal']


    return mktval_df.sort_values(by='Date')

In [18]:
def get_fund_name2code():  # Done
    ''' Fund names '''
    return {"Columbia": "CAM",
            "Litman Gregory": "LG",
            "The Arbitrage Credit Opportunities Fund": "TACO",
            "The Arbitrage Event-Driven Fund": "AED",
            "The Arbitrage Fund": "ARB",
            "The Arbitrage Tactical Equity Fund": "TAQ",
            "TransAmerica": "TAF",
            "WIC Arbitrage Partners": "WIC",
            "Water Island Event-Driven Fund": "WED",
            "Water Island Capital Lev Arb Fund": "LEV",
            "WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL": "MACO",
            "Morningstar Alternatives Fund": "MALT"
           }

def get_fund_code2name():
    return {v: k for (k, v) in get_fund_name2code().iteritems()}

In [19]:
def get_tradegroups_snapshot():
    # region query
    query = "SELECT Fund, Sleeve, TradeGroup, Analyst, LongShort, InceptionDate, EndDate, Status," \
            "`Metrics in Bet JSON`,`Metrics in Bet notes JSON`,`Metrics in NAV JSON`,`Metrics in NAV notes JSON` " \
            "FROM " + DB_NAME + ".tradegroups_snapshot2"
    # endregion

    res = wic_optimized_execute(query)
    cols = ['Fund', 'Sleeve', 'TradeGroup', 'Analyst', 'LongShort', 'InceptionDate', 'EndDate', 'Status',
            'Metrics in Bet JSON', 'Metrics in Bet notes JSON', 'Metrics in NAV JSON', 'Metrics in NAV notes JSON']
    return pd.DataFrame(list(res), columns=cols)

In [20]:
def df2row(pivot_col, df):
    dfcols = [c for c in df.columns if c != pivot_col]
    cols = [colname + '|' + period for (colname, period) in itertools.product(dfcols, df[pivot_col])]
    cols2vals = {c:None for c in cols}

    for idx in df.index:
        row = df.loc[idx]
        pivot = row[pivot_col]
        for cln in dfcols:
            key = cln + '|' + pivot
            cols2vals[key] = row[cln]
    res = pd.Series()
    for cln in cols:
        res[cln] = cols2vals[cln]
    return res

def json2row(json):
    df = pd.read_json(json)
    return df2row('Period', df)

In [21]:
def get_tradegroups_attribution_to_fund_nav():
    today = datetime.datetime.today()

    df = get_tradegroups_snapshot()

    df['EndDate'] = df['EndDate'].apply(lambda x: x if x is None else pd.to_datetime(x).strftime('%m/%d/%y'))
    df['InceptionDate'] = df['InceptionDate'].apply(lambda x: x if x is None else pd.to_datetime(x).strftime('%m/%d/%y'))
    #Do not want tradegroups closed before the start of the current year
    #df_reduced = df[df["EndDate"].dt.year >= today.year] #Takes out deals like cash that don't have date- wrong

    metrics2include = [('P&L(bps)', 'ITD'), ('P&L($)', 'ITD'),
                       ('P&L(bps)', 'YTD'), ('P&L($)', 'YTD'),
                       ('P&L(bps)', 'QTD'), ('P&L($)', 'QTD'),
                       ('P&L(bps)', 'MTD'), ('P&L($)', 'MTD'),
                       ('P&L(bps)', '30D'), ('P&L($)', '30D'),
                       ('P&L(bps)', '5D'), ('P&L($)', '5D'),
                       ('P&L(bps)', '1D'), ('P&L($)', '1D')]

    metric2display_name = {'P&L(bps)': '', 'P&L($)': ''}
    metric2unit = {'P&L(bps)': 'bps', 'P&L($)': '$'}

    # unjsonify metrics, and append columns
    metrics_df = pd.DataFrame([json2row(json) for json in df['Metrics in NAV JSON']])
    metrics_df.index = df.index

    #cln2decimal_pts = {}
    #colnames_to_sum = []
    #display_columns = []
    for (metric, period) in metrics2include:
        unit = metric2unit[metric]
        disp_name = metric2display_name[metric]
        display_colname = disp_name + ' ' + period + '(' + unit + ')'
        df[display_colname] = metrics_df[metric + '|' + period]
        #if unit == '$':
            #cln2decimal_pts[display_colname] = 0
            #colnames_to_sum.append(display_colname)
        #if unit == 'bps':
            #cln2decimal_pts[display_colname] = 1
            #colnames_to_sum.append(display_colname)
        #if unit == '%':
            #cln2decimal_pts[display_colname] = 2
        #display_columns.append(display_colname)

    del df['Metrics in NAV JSON']; del df['Metrics in NAV notes JSON'];
    del df['Metrics in Bet JSON']; del df['Metrics in Bet notes JSON'];
    del df['Analyst']

    sleeve2code = {'Merger Arbitrage': 'M&A', 'Equity Special Situations': 'ESS',
                   'Opportunistic' : 'OPP', 'Forwards':'FWD', 'Credit Opportunities':'CREDIT'}

    df['Sleeve'] = df['Sleeve'].apply(lambda x: sleeve2code[x] if x in sleeve2code else x)
    df = df[(~pd.isnull(df[' YTD($)']))]  # don't show null ytds. i.e. tradegroups closed before year started
    df['Date'] = today.strftime('%Y-%m-%d')

    base_cols = ['Date', 'Fund', 'Sleeve', 'TradeGroup', 'LongShort', 'InceptionDate', 'EndDate', 'Status']
    bps_cols = [' ITD(bps)', ' YTD(bps)', ' QTD(bps)', ' MTD(bps)', ' 30D(bps)', ' 5D(bps)', ' 1D(bps)']  
    dollar_cols = [' ITD($)', ' YTD($)', ' QTD($)', ' MTD($)', ' 30D($)', ' 5D($)', ' 1D($)']
    bps_df = df[base_cols+bps_cols].sort_values(by=' YTD(bps)')
    bps_df.rename(columns={' ITD(bps)': 'ITD(bps)', ' YTD(bps)': 'YTD(bps)', ' QTD(bps)': 'QTD(bps)',
                           ' MTD(bps)': 'MTD(bps)', ' 30D(bps)': '30D(bps)', ' 5D(bps)': '5D(bps)',
                           ' 1D(bps)': '1D(bps)'}, inplace=True)
    dollar_df = df[base_cols+dollar_cols].sort_values(by=' YTD($)')
    dollar_df.rename(columns={' ITD($)': 'ITD($)', ' YTD($)': 'YTD($)', ' QTD($)': 'QTD($)',
                              ' MTD($)': 'MTD($)', ' 30D($)': '30D($)', ' 5D($)': '5D($)',
                              ' 1D($)': '1D($)'}, inplace=True)

    return bps_df, dollar_df

In [24]:
#TradeGroup Attribution to Fund NAV Page
x = get_tradegroups_attribution_to_fund_nav() 
#Produces two dataframes: one for bps and one for dollar
unique_tradegroups = x[0]['TradeGroup'].unique()
limit_to_tradegroups = list(unique_tradegroups)
unique_funds = x[0]['Fund'].unique()
limit_to_funds = list(unique_funds)

In [26]:
fund_and_tradegroup_combos = x[0][["Fund", "TradeGroup"]]
fund_and_tradegroup_combos = fund_and_tradegroup_combos.reset_index()
del fund_and_tradegroup_combos['index']
fund_and_tradegroup_combos

In [22]:
PCVH = get_position_calculated_values_history(limit_to_tradegroups=['ANDX - MPLX'], fund_code='LEV')

PCVH retrieval completed....


In [23]:
PCVH

,Date,FundCode,TradeGroup,TradeGroup_Tradar_Name,Sleeve,Bucket,TradeGroupId,Ticker,SecType,MarketCapCategory,...,Capital(%),BaseCaseNavImpact,OutlierNavImpact,Qty,LongShort,Sector,Industry,Fund_NAV,Equity_Risk_Exp,DealDuration
0,2019-05-09,LEV,ANDX - MPLX,ANDX - MPLX,Merger Arbitrage,Optimized,94900.0,ANDX US,EQ,Mid Cap,...,1.5036,0.45,1.04000,2455.0,Long,Energy,"Oil, Gas & Consumable Fuels",5691731.470000,84997.511812,0M-3M
1,2019-05-09,LEV,ANDX - MPLX,ANDX - MPLX,Merger Arbitrage,Optimized,94900.0,MPLX US,EQ,Large Cap,...,NaN,-0.68,-1.33000,-2786.0,Long,Energy,"Oil, Gas & Consumable Fuels",5691731.470000,-85558.060000,0M-3M
2,2019-05-10,LEV,ANDX - MPLX,ANDX - MPLX,Merger Arbitrage,Optimized,94900.0,ANDX US,EQ,Mid Cap,...,1.5390,0.42,1.02000,2455.0,Long,Energy,"Oil, Gas & Consumable Fuels",5670723.470000,86368.368653,0M-3M
3,2019-05-10,LEV,ANDX - MPLX,ANDX - MPLX,Merger Arbitrage,Optimized,94900.0,MPLX US,EQ,Large Cap,...,NaN,-0.65,-1.31000,-2786.0,Long,Energy,"Oil, Gas & Consumable Fuels",5670723.470000,-87313.240000,0M-3M


In [24]:
SPREADS_HISTORY_DF = get_spread_history()

In [26]:
TG_PNL_DF = get_tradegroups_pnl_cache(fundcode='LEV', tg='ANDX - MPLX')

In [28]:
FUND_NAV_DF = get_NAV_df()

In [30]:
SECURITIES_PNL_DF = get_securities_pnl_by_tradegroup_and_fund(limit_to_tradegroups=['ANDX - MPLX'],
                                                              limit_to_funds=['LEV'], rollup_pnl=True)

0.0499999523163


In [31]:
SECURITIES_PNL_DF

,Date,Fund,TradeGroup,Ticker,SecType,Total P&L,Cumulative Total P&L
0,2019-05-09,LEV,ANDX - MPLX,ANDX US,Equity,302.2105,None
1,2019-05-10,LEV,ANDX - MPLX,ANDX US,Equity,1693.9500,None
2,2019-05-09,LEV,ANDX - MPLX,MPLX US,Equity,-499.6194,None
3,2019-05-10,LEV,ANDX - MPLX,MPLX US,Equity,-1755.1800,None
4,2019-05-09,LEV,ANDX - MPLX,USD,Currencies,0.0000,None
5,2019-05-10,LEV,ANDX - MPLX,USD,Currencies,0.0000,None


In [191]:
#def get_current_tradegroup():
#    query = "SELECT `Date`, Fund, TradeGroup FROM " + DB_NAME + ".tradegroup_attribution_to_fund_nav_dollar " \
#            "WHERE `Date` = '2018-05-23'"
#    
#    res = wic_optimized_execute(query)
#    cols=['Date', 'Fund', 'TradeGroup']
#    return pd.DataFrame(list(res), columns=cols)

In [192]:
#f = get_current_tradegroup()
#f

In [ ]:
#It is best to fill in the history by fund as the securities_pnl query takes too long to do all funds at once

In [32]:
def daily_pnl_df(fund_code, SECURITIES_PNL_DF, TG_PNL_DF, PCVH, FUND_NAV_DF):
    
    start = time.time()
    SECURITIES_PNL_DF['TradeGroup'] = SECURITIES_PNL_DF['TradeGroup'].apply(lambda x: x.upper())
    TG_PNL_DF['TradeGroup'] = TG_PNL_DF['TradeGroup'].apply(lambda x: x.upper())
    PCVH['TradeGroup'] = PCVH['TradeGroup'].apply(lambda x: x.upper())
    
    tradegroups = SECURITIES_PNL_DF['TradeGroup'].unique()
    #limit_to_tradegroups = list(fund_tg_combos['TradeGroup'].unique())
    #limit_to_funds = list(fund_tg_combos['Fund'].unique())
    final_pnl_df = pd.DataFrame()
    final_ticker_pnl_df = pd.DataFrame()
    for i in range(0, len(tradegroups)):
        bps_dollar_pnl_df = pd.DataFrame()
        bps_dollar_ticker_pnl_df = pd.DataFrame()
        tradar_tg_name = tradegroups[i]
        print(tradar_tg_name)
        fund = fund_code
        
        tg_pcvh = PCVH[(PCVH.TradeGroup == tradar_tg_name) & (PCVH.FundCode == fund)].copy()
        fund_nav_df = FUND_NAV_DF[FUND_NAV_DF.FundCode == fund].copy()
        securities_pnl_df = SECURITIES_PNL_DF[(SECURITIES_PNL_DF.TradeGroup == tradar_tg_name)].copy()
        tg_pnl_df = TG_PNL_DF[(TG_PNL_DF.TradeGroup == tradar_tg_name) & (TG_PNL_DF.Fund == fund)].copy()

        options_pnl_df = securities_pnl_df[securities_pnl_df['SecType'] ==
                                           'Option'][['Date', 'Total P&L']].groupby('Date').sum().reset_index().copy()
        options_pnl_df = options_pnl_df.iloc[np.trim_zeros(options_pnl_df['Total P&L']).index].copy()

        if len(options_pnl_df) > 0:
            options_pnl_df = options_pnl_df.rename(columns={'Total P&L': 'Options_PnL_Dollar'})
            options_pnl_df['Fund'] = fund
            options_pnl_df['TradeGroup'] = tradar_tg_name
            options_pnl_df = pd.merge(options_pnl_df, fund_nav_df, how='inner', on=['Date'])
            options_pnl_df['Shifted Capital'] = options_pnl_df['NAV'].shift(1)
            if pd.isnull(options_pnl_df['Shifted Capital'].iloc[0]):
                options_pnl_df.loc[0, 'Shifted Capital'] = options_pnl_df['NAV'].iloc[0]
            options_pnl_df['Shifted Capital'] = options_pnl_df['Shifted Capital'].apply(lambda x: np.nan if x == 0 else x)
            options_pnl_df['Shifted Forward-Filled Capital'] = options_pnl_df['Shifted Capital'].ffill()
            options_pnl_df['Options_PnL_bps'] = 1e4*(options_pnl_df["Options_PnL_Dollar"]/
                                                     options_pnl_df["Shifted Forward-Filled Capital"]).replace([np.inf,
                                                                                                                -np.inf],
                                                                                                               np.nan)
            if bps_dollar_pnl_df.empty:
                bps_dollar_pnl_df = bps_dollar_pnl_df.append(options_pnl_df[['Date', 'Fund', 'TradeGroup',
                                                                         'Options_PnL_Dollar', 'Options_PnL_bps']], sort=True)
            else:
                bps_dollar_pnl_df = pd.merge(bps_dollar_pnl_df, options_pnl_df[['Date', 'Fund', 'TradeGroup',
                                                                         'Options_PnL_Dollar', 'Options_PnL_bps']],
                                             how='outer', on=['Date', 'Fund', 'TradeGroup'])

        #P&L Breakdown by Ticker -- to be kept in a separate db
        tg_securities = securities_pnl_df[securities_pnl_df['SecType'] != 'Option']['Ticker'].unique()
        for ticker in tg_securities:
            tkr_pnl_df = securities_pnl_df[securities_pnl_df['Ticker'] == ticker].copy()
            tkr_pnl_df = tkr_pnl_df.loc[np.trim_zeros(tkr_pnl_df['Total P&L']).index].copy()
            if len(tkr_pnl_df) == 0:
                continue
            tkr_pnl_df = tkr_pnl_df.rename(columns={'Total P&L': 'Ticker_PnL_Dollar'})
            tkr_pnl_df['Fund'] = fund
            tkr_pnl_df['TradeGroup'] = tradar_tg_name
            tkr_pnl_df = pd.merge(tkr_pnl_df, fund_nav_df, how='inner', on=['Date'])
            tkr_pnl_df['Shifted Capital'] = tkr_pnl_df['NAV'].shift(1)
            if pd.isnull(tkr_pnl_df['Shifted Capital'].iloc[0]):
                tkr_pnl_df.loc[0, 'Shifted Capital'] = tkr_pnl_df['NAV'].iloc[0]
            tkr_pnl_df['Shifted Capital'] = tkr_pnl_df['Shifted Capital'].apply(lambda x: np.nan if x == 0 else x)
            tkr_pnl_df['Shifted Forward-Filled Capital'] = tkr_pnl_df['Shifted Capital'].ffill()
            tkr_pnl_df['Ticker_PnL_bps'] = 1e4*(tkr_pnl_df["Ticker_PnL_Dollar"]/
                                                     tkr_pnl_df["Shifted Forward-Filled Capital"]).replace([np.inf,
                                                                                                            -np.inf],
                                                                                                            np.nan)  
            bps_dollar_ticker_pnl_df = bps_dollar_ticker_pnl_df.append(tkr_pnl_df[['Date', 'Fund', 'TradeGroup',
                                                                         'Ticker_PnL_Dollar', 'Ticker_PnL_bps', 'Ticker']])

        final_ticker_pnl_df = final_ticker_pnl_df.append(bps_dollar_ticker_pnl_df, sort=True)

        #Daily PnL  
        daily_pnl_df = tg_pnl_df[['Date', 'Fund', 'TradeGroup', 'Total P&L']]
        if len(daily_pnl_df) > 0:
            daily_pnl_df = daily_pnl_df.rename(columns={'Total P&L': 'Daily_PnL_Dollar'})
            daily_pnl_df['Fund'] = fund
            daily_pnl_df['TradeGroup'] = tradar_tg_name
            daily_pnl_df = pd.merge(daily_pnl_df, fund_nav_df, how='inner', on=['Date'])
            daily_pnl_df['Shifted Capital'] = daily_pnl_df['NAV'].shift(1)
            if pd.isnull(daily_pnl_df['Shifted Capital'].iloc[0]):
                daily_pnl_df.loc[0, 'Shifted Capital'] = daily_pnl_df['NAV'].iloc[0]
            daily_pnl_df['Shifted Capital'] = daily_pnl_df['Shifted Capital'].apply(lambda x: np.nan if x == 0 else x)
            daily_pnl_df['Shifted Forward-Filled Capital'] = daily_pnl_df['Shifted Capital'].ffill()
            daily_pnl_df['Daily_PnL_bps'] = 1e4*(daily_pnl_df["Daily_PnL_Dollar"]/
                                                     daily_pnl_df["Shifted Forward-Filled Capital"]).replace([np.inf, -np.inf],
                                                                                                             np.nan)  
            if bps_dollar_pnl_df.empty:
                bps_dollar_pnl_df = bps_dollar_pnl_df.append(daily_pnl_df[['Date', 'Fund', 'TradeGroup',
                                                                         'Daily_PnL_Dollar', 'Daily_PnL_bps']], sort=True)
            else:
                bps_dollar_pnl_df = pd.merge(bps_dollar_pnl_df, daily_pnl_df[['Date', 'Fund', 'TradeGroup',
                                                                         'Daily_PnL_Dollar', 'Daily_PnL_bps']], how='outer',
                                             on=['Date', 'Fund', 'TradeGroup'])

        cum_pnl = pd.DataFrame()
        rolling_vol_df = pd.DataFrame()
        if len(tg_pcvh) > 0:
                tg_capital_df = mktval_df(tg_pcvh)
                roc_df = pd.merge(tg_pnl_df, tg_capital_df[['Date', 'Bet GrossMktVal']], how='inner', on=['Date'])
                if len(roc_df) > 1:
                    roc_df['Shifted Capital'] = roc_df['Bet GrossMktVal'].shift(1)
                    if pd.isnull(roc_df['Shifted Capital'].iloc[0]):
                        roc_df.loc[0, 'Shifted Capital'] = roc_df['Bet GrossMktVal'].iloc[0]
                    roc_df['Shifted Capital'] = roc_df['Shifted Capital'].apply(lambda x: np.nan if x == 0 else x)
                    roc_df['Shifted Forward-Filled Capital'] = roc_df['Shifted Capital'].ffill()
                    roc_df['PnL_Over_Cap_bps'] = 1e4*(roc_df["Total P&L"]/
                                               roc_df["Shifted Forward-Filled Capital"]).replace([np.inf, -np.inf],
                                                                                                 np.nan) 

                    #roc_df['Cumulative P&L (%)'] = roc_df['Cumulative P&L bps (ffiled)'].apply(lambda x: x/100.0)
                    roc_df['Rolling_30D_PnL_Vol'] = math.sqrt(252)*(roc_df['PnL_Over_Cap_bps']/100.0).rolling(window=30).std()
                    if len(roc_df['Rolling_30D_PnL_Vol'].dropna()) > 0:
                        rolling_vol_df[['Date', 'Fund', 'TradeGroup', 'Rolling_30D_PnL_Vol']] = roc_df[
                            ["Date", "Fund", "TradeGroup", "Rolling_30D_PnL_Vol"]].dropna()

                        if bps_dollar_pnl_df.empty:
                            bps_dollar_pnl_df = bps_dollar_pnl_df.append(rolling_vol_df[['Date', 'Fund', 'TradeGroup',
                                                                                         'Rolling_30D_PnL_Vol']], sort=True)
                        else:
                            bps_dollar_pnl_df = bps_dollar_pnl_df.merge(rolling_vol_df[['Date', 'Fund',
                                                                                        'TradeGroup',
                                                                                        'Rolling_30D_PnL_Vol']],
                                                                        how='outer', on=['Date', 'Fund', 'TradeGroup'])

                    if bps_dollar_pnl_df.empty:
                        bps_dollar_pnl_df = bps_dollar_pnl_df.append(roc_df[['Date', 'Fund', 'TradeGroup'
                                                                         'PnL_Over_Cap_bps']], sort=True)
                    else:
                        bps_dollar_pnl_df = bps_dollar_pnl_df.merge(roc_df[['Date', 'Fund', 'TradeGroup', 'PnL_Over_Cap_bps']],
                                                                    how='outer', on=['Date', 'Fund', 'TradeGroup'])
        
        final_pnl_df = final_pnl_df.append(bps_dollar_pnl_df)
    
    end = time.time()
    print(end-start)
    return final_pnl_df, final_ticker_pnl_df
        

In [33]:
#pd.set_option('display.max_rows', 1500)

In [34]:
f = 'LEV'
x = daily_pnl_df(f, SECURITIES_PNL_DF, TG_PNL_DF, PCVH, FUND_NAV_DF)

ANDX - MPLX
0.149000167847


In [38]:
a = x[0]
a = a[a.Date == '2019-05-09']
a

,Daily_PnL_Dollar,Daily_PnL_bps,Date,Fund,TradeGroup,PnL_Over_Cap_bps
0,-197.4089,-0.346835,2019-05-09,LEV,ANDX - MPLX,-23.066827


In [39]:
b = x[1]
b = b[b.Date == '2019-05-09']
b

,Date,Fund,Ticker,Ticker_PnL_Dollar,Ticker_PnL_bps,TradeGroup
0,2019-05-09,LEV,ANDX US,302.2105,0.530964,ANDX - MPLX
0,2019-05-09,LEV,MPLX US,-499.6194,-0.877799,ANDX - MPLX


In [41]:
#from sqlalchemy import create_engine
#WIC_DB_HOST = 'wic-risk-database.cwi02trt7ww1.us-east-1.rds.amazonaws.com'
#DB_USER = 'root'
#DB_PASSWORD = 'waterislandcapital'
#DB_NAME = 'dev_wic_db'
#engine = create_engine("mysql://" + DB_USER + ":" + DB_PASSWORD + "@" + WIC_DB_HOST + "/dev_wic_db")
#con = engine.connect()
#b.to_sql(con=con, schema='dev_wic_db', name='ticker_pnl_breakdown', if_exists='append', chunksize=1000, index=False)
#con.close()

In [204]:
a.to_csv('C:/Users/aduprey/Desktop/TradeGroups_Data/ARB_pnl_overall.csv')
b.to_csv('C:/Users/aduprey/Desktop/TradeGroups_Data/ARB_pnl_ticker.csv')

In [42]:
def get_exposure_spreads_nav_df(fund_c, PCVH, FUND_NAV_DF, SPREAD_HISTORY_DF):
    start = time.time()
    PCVH['TradeGroup'] = PCVH['TradeGroup'].apply(lambda x: x.upper())
    SPREAD_HISTORY_DF['TradeGroup'] = SPREAD_HISTORY_DF['TradeGroup'].apply(lambda x: x.upper())
    overall_df = pd.DataFrame()
    fund = fund_c
    PCVH = PCVH[PCVH.FundCode == fund]
    tg_combos = PCVH['TradeGroup'].unique()
    for i in range(0, len(tg_combos)):
        overall_exp_df = pd.DataFrame()
        tradar_tg_name = tg_combos[i]
        #fund = fund_tg_combos['Fund'][i]

        tg_pcvh = PCVH[(PCVH.TradeGroup == tradar_tg_name)].copy()
        fund_nav_df = FUND_NAV_DF[FUND_NAV_DF.FundCode == fund].copy()
        spreads_history_df = SPREADS_HISTORY_DF[SPREADS_HISTORY_DF.TradeGroup_TradarName == tradar_tg_name].copy()
        
        # Capital AS (%) OF NAV TIME SERIES #
        cap_df = pd.DataFrame()
        capital_df = tg_pcvh[tg_pcvh['AlphaHedge']
                             .isin(['Alpha',
                                    'Alpha Hedge'])][['Date',
                                                      'NetMktVal']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index()
        if len(capital_df) > 0:
            capital_df = pd.merge(capital_df, fund_nav_df, how='inner', on=['Date'])
            capital_df['CapitalAsPctOfNAV'] = 100.0*(capital_df['NetMktVal'].astype(float)
                                                 /capital_df['NAV'].astype(float))
    
            cap_df['Date'] = capital_df['Date']
            cap_df['Capital_Percent_of_NAV'] = capital_df['CapitalAsPctOfNAV']
            cap_df['Fund'] = fund
            cap_df['TradeGroup'] = tradar_tg_name
            
            if overall_exp_df.empty:
                overall_exp_df = overall_exp_df.append(cap_df)
            else:
                overall_exp_df = overall_exp_df.merge(cap_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])
        
        # NETEXP (% OF NAV) #
        net_exp_df = pd.DataFrame()
        tot_net_exp_df = tg_pcvh[['Date', 'Exposure_Net']].groupby('Date').sum().reset_index()
        tot_net_exp_df = tot_net_exp_df.rename(columns={'Exposure_Net':'Total Exposure'}).sort_values(by='Date')
        tot_net_exp_df = pd.merge(tot_net_exp_df, fund_nav_df, how='inner', on=['Date'])
        tot_net_exp_df['NetExpAsPctOfNAV'] = 100.0*(tot_net_exp_df['Total Exposure'].astype(float)
                                                    /tot_net_exp_df['NAV'].astype(float))
        if len(tot_net_exp_df) > 0:
            net_exp_df['Date'] = tot_net_exp_df['Date']
            net_exp_df['NetExp_Percent_of_NAV'] = tot_net_exp_df['NetExpAsPctOfNAV']
            net_exp_df['Fund'] = fund
            net_exp_df['TradeGroup'] = tradar_tg_name
            
            if overall_exp_df.empty:
                overall_exp_df = overall_exp_df.append(net_exp_df)
            else:
                overall_exp_df = overall_exp_df.merge(net_exp_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])

        # GROSSEXP (% OF NAV) #
        gross_exp_df = pd.DataFrame()
        tot_gross_exp_df = tg_pcvh[['Date', 'Exposure_Net']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index()
        tot_gross_exp_df = tot_gross_exp_df.rename(columns={'Exposure_Net':
                                                            'Total Gross Exposure'}).sort_values(by='Date')
        tot_gross_exp_df = pd.merge(tot_gross_exp_df, fund_nav_df, how='inner', on=['Date'])
        tot_gross_exp_df['GrossExpAsPctOfNAV'] = 100.0*(tot_gross_exp_df['Total Gross Exposure'].astype(float)
                                                        /tot_gross_exp_df['NAV'].astype(float))
        if len(tot_gross_exp_df) > 0:
            gross_exp_df['Date'] = tot_gross_exp_df['Date']
            gross_exp_df['GrossExp_Percent_of_NAV'] = tot_gross_exp_df['GrossExpAsPctOfNAV']
            gross_exp_df['Fund'] = fund
            gross_exp_df['TradeGroup'] = tradar_tg_name
            
            if overall_exp_df.empty:
                overall_exp_df = overall_exp_df.append(gross_exp_df)
            else:
                overall_exp_df = overall_exp_df.merge(gross_exp_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])
           
        hedge_exp_df = pd.DataFrame()
        if len(tg_pcvh) > 0:
            alpha_net_exp = tg_pcvh[tg_pcvh['AlphaHedge'] ==
                                    'Alpha'][['Date', 'Exposure_Net']].groupby('Date').sum().reset_index()
            alphahedge_net_exp = tg_pcvh[tg_pcvh['AlphaHedge'] ==
                                         'Alpha Hedge'][['Date', 'Exposure_Net']].groupby('Date').sum().reset_index()
            hedge_net_exp = tg_pcvh[tg_pcvh['AlphaHedge'] ==
                                    'Hedge'][['Date', 'Exposure_Net']].groupby('Date').sum().reset_index()

            if len(alpha_net_exp) > 0:
                alpha_net_exp_df = pd.DataFrame()
                alpha_net_exp = pd.merge(alpha_net_exp, fund_nav_df, how='inner', on=['Date'])
                alpha_net_exp['Exposure_Net'] = 100.0*(alpha_net_exp['Exposure_Net'].astype(float)
                                                       /alpha_net_exp['NAV'].astype(float))
                alpha_net_exp_df['Date'] = alpha_net_exp['Date']
                alpha_net_exp_df['Alpha_Exposure'] = alpha_net_exp['Exposure_Net']
                alpha_net_exp_df['Fund'] = fund
                alpha_net_exp_df['TradeGroup'] = tradar_tg_name
                
                if overall_exp_df.empty:
                    overall_exp_df = overall_exp_df.append(alpha_net_exp_df)
                else:
                    overall_exp_df = overall_exp_df.merge(alpha_net_exp_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])
            
            if len(alphahedge_net_exp) > 0:
                alphahedge_net_exp_df = pd.DataFrame()
                alphahedge_net_exp = pd.merge(alphahedge_net_exp, fund_nav_df, how='inner', on=['Date'])
                alphahedge_net_exp['Exposure_Net'] = 100.0*(alphahedge_net_exp['Exposure_Net'].astype(float)
                                                            /alphahedge_net_exp['NAV'].astype(float))
                alphahedge_net_exp_df['Date'] = alphahedge_net_exp['Date']
                alphahedge_net_exp_df['AlphaHedge_Exposure'] = alphahedge_net_exp['Exposure_Net']
                alphahedge_net_exp_df['Fund'] = fund
                alphahedge_net_exp_df['TradeGroup'] = tradar_tg_name
            
                if overall_exp_df.empty:
                    overall_exp_df = overall_exp_df.append(alphahedge_net_exp_df)
                else:
                    overall_exp_df = overall_exp_df.merge(alphahedge_net_exp_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])

            if len(hedge_net_exp) > 0:
                hedge_net_exp_df = pd.DataFrame()
                hedge_net_exp = pd.merge(hedge_net_exp, fund_nav_df, how='inner', on=['Date'])
                hedge_net_exp['Exposure_Net'] = 100.0*(hedge_net_exp['Exposure_Net'].astype(float)
                                                       /hedge_net_exp['NAV'].astype(float))
                hedge_net_exp_df['Date'] = hedge_net_exp['Date']
                hedge_net_exp_df['Hedge_Exposure'] = hedge_net_exp['Exposure_Net']
                hedge_net_exp_df['Fund'] = fund
                hedge_net_exp_df['TradeGroup'] = tradar_tg_name
                
                if overall_exp_df.empty:
                    overall_exp_df = overall_exp_df.append(hedge_net_exp_df)
                else:
                    overall_exp_df = overall_exp_df.merge(hedge_net_exp_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])
        
        #SPREAD AS PERCENT#
        spread = pd.DataFrame()
        if tradar_tg_name in spreads_history_df['TradeGroup_TradarName'].unique():
            spreads_ts_df = spreads_history_df[spreads_history_df['TradeGroup_TradarName'] ==
                                               tradar_tg_name][['Date', 'Spread(%)']].sort_values(by='Date')
            spreads_ts_df = spreads_ts_df.rename(columns={'Spread(%)':'SpreadAsPct'}).sort_values(by='Date')
            spreads_ts_df['SpreadAsPct'] = spreads_ts_df['SpreadAsPct'].astype(float)
            
            spread['Date'] = spreads_ts_df['Date']
            spread['Spread_as_Percent'] = spreads_ts_df['SpreadAsPct']
            spread['Fund'] = fund
            spread['TradeGroup'] = tradar_tg_name
            
            if overall_exp_df.empty:
                overall_exp_df = overall_exp_df.append(spread)
            else:
                overall_exp_df = overall_exp_df.merge(spread, how='outer', on=['Date', 'Fund', 'TradeGroup'])
                
        overall_df = overall_df.append(overall_exp_df)  
    
    end = time.time()
    print(end - start)
    return overall_df

In [43]:
overall_df = get_exposure_spreads_nav_df('LEV', PCVH, FUND_NAV_DF, SPREADS_HISTORY_DF)

0.111999988556


In [44]:
overall_df

,Date,Capital_Percent_of_NAV,Fund,TradeGroup,NetExp_Percent_of_NAV,GrossExp_Percent_of_NAV,Alpha_Exposure,Hedge_Exposure,Spread_as_Percent
0,2019-05-09,1.503607,LEV,ANDX - MPLX,0.000408,3.006807,1.503607,-1.503199,0.670177
1,2019-05-09,1.503607,LEV,ANDX - MPLX,0.000408,3.006807,1.503607,-1.503199,1.044904
2,2019-05-10,1.539050,LEV,ANDX - MPLX,-0.000670,3.078769,1.539050,-1.539720,1.097250
3,2019-05-08,NaN,LEV,ANDX - MPLX,NaN,NaN,NaN,NaN,-6.794764
